In [1]:
import pandas as pd

edges = pd.read_csv("edges.csv", index_col=0)


C:\Users\ersin\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ersin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
C:\Users\ersin\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
investors = pd.read_csv("investors.csv", index_col=0)
stocks = pd.read_csv("stocks.csv", index_col=0)


In [3]:
investors.shape , stocks.shape

((8840, 2), (11679, 6))

In [4]:
edges.shape

(24034481, 8)

In [5]:
edges.head()

,cikname,name,quarter,changeCat,delay_cat,volatility_category,volume_category,direction
0,FRED ALGER MANAGEMENT INC,AAC HOLDINGS INC,2015-06-30,Buy,on time,NaN,NaN,stock_to_investor
1,FRED ALGER MANAGEMENT INC,AAR CORP,2011-12-31,Large Sale,on time,very low,high,investor_to_stock
2,FRED ALGER MANAGEMENT INC,AAR CORP,2012-03-31,Significant Sale,on time,very low,low,investor_to_stock
3,FRED ALGER MANAGEMENT INC,ADT CORP,2012-12-31,Significant Buy,on time,NaN,NaN,stock_to_investor
4,FRED ALGER MANAGEMENT INC,ADT CORP,2013-03-31,Large Buy,on time,NaN,NaN,stock_to_investor


In [6]:
import pandas as pd

# Assuming 'edges', 'investors', and 'stocks' are your DataFrames

# Merge edges and investors on 'cikname'
merged_df = pd.merge(edges, investors, on='cikname', how='left')

# Merge the result with stocks on 'name'
final_merged_df = pd.merge(merged_df, stocks, on='name', how='left')


In [1]:
final_merged_df.tail()

NameError: name 'final_merged_df' is not defined

In [8]:
final_merged_df.shape

(24596386, 14)

In [12]:
final_merged_df['cikname'] = final_merged_df['cikname'] + '_investor'

# Add '_stock' suffix to each record in the 'name' column
final_merged_df['name'] = final_merged_df['name'] + '_stock'

In [13]:
final_merged_df.head()

,cikname,name,quarter,changeCat,delay_cat,volatility_category,volume_category,direction,investor_rank,ticker,equityType,sicSector,sicIndustry,ticker_classification,edge_id
0,FRED ALGER MANAGEMENT INC_investor,AAC HOLDINGS INC_stock,2015-06-30,Buy,on time,NaN,NaN,stock_to_investor,large_investor,AACH,Domestic Common Stock,Services,Services-Specialty Outpatient Facilities Nec,mid,FRED ALGER MANAGEMENT INCAAC HOLDINGS INC2015-...
1,FRED ALGER MANAGEMENT INC_investor,AAR CORP_stock,2011-12-31,Large Sale,on time,very low,high,investor_to_stock,large_investor,AIR,Domestic Common Stock,Manufacturing,Aircraft & Parts,mid,FRED ALGER MANAGEMENT INCAAR CORP2011-12-31
2,FRED ALGER MANAGEMENT INC_investor,AAR CORP_stock,2012-03-31,Significant Sale,on time,very low,low,investor_to_stock,large_investor,AIR,Domestic Common Stock,Manufacturing,Aircraft & Parts,mid,FRED ALGER MANAGEMENT INCAAR CORP2012-03-31
3,FRED ALGER MANAGEMENT INC_investor,ADT CORP_stock,2012-12-31,Significant Buy,on time,NaN,NaN,stock_to_investor,large_investor,ADT1,Domestic Common Stock,Services,Services-Detective Guard & Armored Car Services,mid,FRED ALGER MANAGEMENT INCADT CORP2012-12-31
4,FRED ALGER MANAGEMENT INC_investor,ADT CORP_stock,2013-03-31,Large Buy,on time,NaN,NaN,stock_to_investor,large_investor,ADT1,Domestic Common Stock,Services,Services-Detective Guard & Armored Car Services,mid,FRED ALGER MANAGEMENT INCADT CORP2013-03-31


In [14]:
import networkx as nx
import pandas as pd

# Assuming your DataFrame is named df

# Preprocess DataFrame to ensure each edge is unique
final_merged_df['edge_id'] = final_merged_df['cikname'] + final_merged_df['name'] + final_merged_df['quarter'].astype(str)

# Create a MultiDiGraph from the DataFrame
G = nx.from_pandas_edgelist(final_merged_df, source='cikname', target='name', edge_attr=True, create_using=nx.MultiGraph)



In [15]:
# Add node attributes for bipartite sets
bipartite_attr = {node: 0 for node in final_merged_df['cikname']}  # Assuming cikname nodes as one set (0)
bipartite_attr.update({node: 1 for node in final_merged_df['name']})  # Assuming name nodes as the other set (1)
nx.set_node_attributes(G, bipartite_attr, 'bipartite')

# Check if the graph is bipartite
print("Graph is bipartite:", nx.is_bipartite(G))


Graph is bipartite: True


In [16]:
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

Number of nodes: 20391
Number of edges: 24596386


In [17]:
for u,v,a in G.edges(data=True):
    print(u,v,a)
    break

FRED ALGER MANAGEMENT INC_investor AAC HOLDINGS INC_stock {'quarter': '2015-06-30', 'changeCat': 'Buy', 'delay_cat': 'on time', 'volatility_category': nan, 'volume_category': nan, 'direction': 'stock_to_investor', 'investor_rank': 'large_investor', 'ticker': 'AACH', 'equityType': 'Domestic Common Stock', 'sicSector': 'Services', 'sicIndustry': 'Services-Specialty Outpatient Facilities Nec', 'ticker_classification': 'mid', 'edge_id': 'FRED ALGER MANAGEMENT INC_investorAAC HOLDINGS INC_stock2015-06-30'}


In [20]:
avg_degree = sum(dict(G.degree()).values()) / len(G)

avg_degree

2412.47471923888

In [21]:
density = nx.density(G)
density

0.11831656298376066

In [22]:
import pickle

# save graph object to file
pickle.dump(G, open('final_multigraph_solved.pickle', 'wb'))

In [ ]:
3.54